In [17]:
from sklearn.ensemble import HistGradientBoostingRegressor
import hopsworks
import os
from datetime import datetime

# Creating the feature view

In [22]:
project = hopsworks.login(api_key_value=os.getenv("HOPSWORKS_API_KEY"))
fs = project.get_feature_store()

fg_price = fs.get_feature_group(
    name="solana_crypto_features",
    version=1,
)

fg_agg_sentiment = fs.get_feature_group(
    name="reddit_aggregated_sentiment_backfill",
    version=1
)

# Create feature view with selected features
selected_features = [
    'timestamp',
    #'close',
    'high', 
    'low',
    'open',
    'volume',
    'close_7d_ma',
    'close_30d_ma',
    'ma_7_30_cross',
    'rsi_14',
    'atr_14',
    'bb_bandwidth',
    'volume_7d_ma',
    'volume_ratio',
    'day_of_week',
    'month'
]

# Join with sentiment features
query = fg_price.select(selected_features + ['close']).join(
    fg_agg_sentiment.select_all(),
    on=['timestamp']
)

feature_view = fs.create_feature_view(
    name="solana_with_sentiment",
    version=2,
    query=query,
    labels=["close"]  # ou votre target
)

2026-01-09 18:08:35,437 INFO: Closing external client and cleaning up certificates.
Connection closed.
2026-01-09 18:08:35,448 INFO: Initializing external client
2026-01-09 18:08:35,449 INFO: Base URL: https://c.app.hopsworks.ai:443
2026-01-09 18:08:36,054 WARNING: UserWarning: The installed hopsworks client version 4.4.2 may not be compatible with the connected Hopsworks backend version 4.2.2. 
To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'



2026-01-09 18:08:37,025 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1279131
2026-01-09 18:08:39,345 WARNING: Ambiguous features detected during query construction.The feature `timestamp` is present in feature groups ['reddit_aggregated_sentiment_backfill version 1', 'solana_crypto_features version 1']. Automatically prefixing features selected using these feature groups with the feature group name.
Feature view created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1279131/fs/1265740/fv/solana_with_sentiment/version/2


# Instanciate the model

In [23]:
start_date_test_data = "2024-01-01"

test_start = datetime.strptime(start_date_test_data, "%Y-%m-%d")

X_train, X_test, y_train, y_test = feature_view.train_test_split(
    test_start=test_start
)

X_train


Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.51s) 
2026-01-09 18:08:59,268 WARNING: VersionWarning: Incremented version to `1`.



,timestamp,high,low,open,volume,close_7d_ma,close_30d_ma,ma_7_30_cross,rsi_14,atr_14,bb_bandwidth,volume_7d_ma,volume_ratio,day_of_week,month,reddit_aggregated_sentiment_backfill_mean_sentiment,reddit_aggregated_sentiment_backfill_count
0,1605484800,2.360211,1.994853,2.043234,22156135,2.055092,1.825444,1,68.548454,0.277274,0.703077,1.458409e+07,1.519199,0,11,0.000000,7.0
1,1607472000,1.674277,1.503676,1.664622,10461119,1.849739,2.026277,0,27.792126,0.189585,0.402118,1.003346e+07,1.042623,2,12,0.090909,11.0
2,1609632000,2.364981,1.799902,1.799902,55073422,1.719928,1.617500,1,64.871286,0.229080,0.556823,2.527390e+07,2.179063,6,1,0.666667,3.0
3,1612396800,6.531011,5.603236,5.731748,108249482,4.906323,3.754754,1,91.401345,0.556394,0.826621,6.736586e+07,1.606889,3,2,0.000000,4.0
4,1613433600,8.980368,7.983557,8.860748,71625205,8.880791,5.719969,1,73.447684,1.013082,1.155841,9.030898e+07,0.793113,1,2,0.000000,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1837,1701820800,66.383621,60.979866,60.979866,3030993886,61.477550,57.001038,1,61.034506,3.570023,0.191287,1.756013e+09,1.726066,2,12,0.263158,19.0
1838,1701993600,75.129791,67.841232,67.846718,3612712432,64.876785,58.886300,1,78.885595,4.226890,0.326349,2.323821e+09,1.554643,4,12,0.222222,18.0
1839,1702598400,79.151070,72.366638,75.636688,2821390652,72.089778,62.753072,1,64.760184,5.689867,0.403961,2.648506e+09,1.065276,4,12,0.388889,18.0
1840,1703548800,121.143997,104.728218,121.084335,6117657160,103.993197,75.669467,1,79.330591,9.648653,0.832903,5.161978e+09,1.185138,1,12,0.235294,17.0


In [25]:
XGBoost_model = HistGradientBoostingRegressor()

XGBoost_model.fit(X_train, y_train)
XGBoost_model.score(X_test, y_test)

2026-01-09 18:09:21,508 WARNING: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.

2026-01-09 18:09:21,510 WARNING: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.

2026-01-09 18:09:21,510 WARNING: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.

2026-01-09 18:09:21,514 WARNING: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.

2026-01-09 18:09:21,514 WARNING: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.

2026-01-09 18:09:21,514 WARNING: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.Spar

0.9752200477249181